
# Projet ROS2 Dashing : Asservissement PID sur Turtlesim (Modèle Bicyclette)

Ce notebook présente un projet complet d'implémentation d'un contrôleur PID basé sur le modèle cinématique bicyclette pour le simulateur turtlesim dans ROS2 Dashing sur Ubuntu 18.04.

## Objectifs du projet

- Créer un package ROS2 Python complet avec ament_python
- Implémenter un contrôleur PID pour suivre une trajectoire
- Utiliser le modèle cinématique bicyclette pour la commande
- Analyser les performances du système avec des outils Python
- Générer tous les livrables nécessaires pour un rapport technique

## Prérequis

- Ubuntu 18.04 LTS
- ROS2 Dashing installé
- Python 3.6+
- Packages Python: matplotlib, numpy, simple-pid, control

## Architecture du système

```
┌─────────────────┐    /turtle1/pose    ┌─────────────────┐
│   turtlesim     │ ──────────────────► │  control_node   │
│   (simulator)   │                     │  (PID + modèle  │
│                 │ ◄─────────────────  │   bicyclette)   │
└─────────────────┘    /turtle1/cmd_vel └─────────────────┘
```



## 1. Installation et Configuration

### 1.1 Installation ROS2 Dashing sur Ubuntu 18.04


In [ ]:

# Commandes d'installation ROS2 Dashing (à exécuter dans le terminal)

# 1. Configuration du locale
sudo apt update && sudo apt install locales
sudo locale-gen en_US en_US.UTF-8
sudo update-locale LC_ALL=en_US.UTF-8 LANG=en_US.UTF-8
export LANG=en_US.UTF-8

# 2. Ajouter les clés GPG et repository ROS2
sudo apt update && sudo apt install curl gnupg2 lsb-release
curl -s https://raw.githubusercontent.com/ros/rosdistro/master/ros.asc | sudo apt-key add -
sudo sh -c 'echo "deb [arch=$(dpkg --print-architecture)] http://packages.ros.org/ros2/ubuntu $(lsb_release -cs) main" > /etc/apt/sources.list.d/ros2-latest.list'

# 3. Installation des packages ROS2
sudo apt update
sudo apt install ros-dashing-desktop python3-argcomplete

# 4. Configuration de l'environnement
echo "source /opt/ros/dashing/setup.bash" >> ~/.bashrc
source ~/.bashrc

# 5. Installation des outils de développement
sudo apt install python3-colcon-common-extensions python3-rosdep
sudo rosdep init
rosdep update

# 6. Installation des dépendances Python
pip3 install simple-pid matplotlib numpy control



### 1.2 Création du Workspace et Package ROS2

Nous allons créer un workspace ROS2 et un package Python pour notre contrôleur PID.


In [ ]:

# Création du workspace ROS2
mkdir -p ~/ros2_ws/src
cd ~/ros2_ws

# Création du package Python avec les dépendances
cd src
ros2 pkg create --build-type ament_python ros2_pid_turtle \
    --dependencies rclpy turtlesim geometry_msgs std_msgs

# Structure du package créée automatiquement:
# ros2_pid_turtle/
# ├── package.xml
# ├── setup.py
# ├── setup.cfg
# ├── resource/
# ├── test/
# └── ros2_pid_turtle/
#     └── __init__.py

print("Package créé avec succès!")



## 2. Structure Détaillée du Package

### 2.1 Arborescence Complète du Projet

Voici l'organisation finale de notre package:



| Fichier/Dossier | Rôle | Description |
|:----------------|:-----|:------------|
| `package.xml` | Métadonnées | Dépendances, version, description |
| `setup.py` | Installation | Entry points, scripts Python |
| `setup.cfg` | Configuration | Configuration setuptools |
| `ros2_pid_turtle/` | Package Python | Code source principal |
| `├── __init__.py` | Init | Initialisation du package |
| `├── control_node.py` | Node principal | Contrôleur PID + modèle bicyclette |
| `├── pid_controller.py` | Classe PID | Implémentation du contrôleur PID |
| `├── bicycle_model.py` | Modèle | Cinématique bicyclette |
| `└── trajectory_generator.py` | Trajectoires | Génération de références |
| `launch/` | Fichiers launch | Démarrage automatique |
| `├── pid_control.launch.py` | Launch principal | Turtlesim + contrôleur |
| `└── analysis.launch.py` | Launch analyse | Enregistrement données |
| `config/` | Configuration | Paramètres YAML |
| `├── pid_params.yaml` | Paramètres PID | Kp, Ki, Kd, consignes |
| `└── bicycle_params.yaml` | Paramètres modèle | Empattement, limites |
| `scripts/` | Scripts utilitaires | Analyse et visualisation |
| `├── plot_results.py` | Visualisation | Graphiques des résultats |
| `└── analyze_performance.py` | Analyse | Calcul des métriques |



### 2.2 Fichier package.xml Complet


In [ ]:

# Contenu du fichier package.xml
package_xml_content = '''<?xml version="1.0"?>
<?xml-model href="http://download.ros.org/schema/package_format3.xsd" schematypens="http://www.w3.org/2001/XMLSchema"?>
<package format="3">
  <name>ros2_pid_turtle</name>
  <version>1.0.0</version>
  <description>
    Contrôleur PID avec modèle cinématique bicyclette pour turtlesim.
    Projet pédagogique d'automatique et robotique.
  </description>

  <!-- Informations du mainteneur -->
  <maintainer email="etudiant@universite.fr">Nom Étudiant</maintainer>
  <license>MIT</license>

  <!-- URLs du projet -->
  <url type="website">https://github.com/user/ros2_pid_turtle</url>
  <url type="bugtracker">https://github.com/user/ros2_pid_turtle/issues</url>
  <url type="repository">https://github.com/user/ros2_pid_turtle</url>

  <!-- Dépendances de build -->
  <buildtool_depend>ament_python</buildtool_depend>

  <!-- Dépendances d'exécution -->
  <exec_depend>rclpy</exec_depend>
  <exec_depend>turtlesim</exec_depend>
  <exec_depend>geometry_msgs</exec_depend>
  <exec_depend>std_msgs</exec_depend>
  <exec_depend>tf2_py</exec_depend>
  <exec_depend>tf2_geometry_msgs</exec_depend>

  <!-- Dépendances de test -->
  <test_depend>ament_copyright</test_depend>
  <test_depend>ament_flake8</test_depend>
  <test_depend>ament_pep257</test_depend>
  <test_depend>python3-pytest</test_depend>

  <export>
    <build_type>ament_python</build_type>
  </export>
</package>'''

# Écriture du fichier
with open('~/ros2_ws/src/ros2_pid_turtle/package.xml', 'w') as f:
    f.write(package_xml_content)

print("Fichier package.xml créé avec succès!")



### 2.3 Fichier setup.py Complet avec Entry Points


In [ ]:

# Contenu du fichier setup.py
setup_py_content = '''import os
from glob import glob
from setuptools import setup

package_name = 'ros2_pid_turtle'

setup(
    name=package_name,
    version='1.0.0',
    packages=[package_name],
    data_files=[
        ('share/ament_index/resource_index/packages',
            ['resource/' + package_name]),
        ('share/' + package_name, ['package.xml']),
        # Installation des fichiers launch
        (os.path.join('share', package_name, 'launch'), 
         glob('launch/*.launch.py')),
        # Installation des fichiers de configuration
        (os.path.join('share', package_name, 'config'), 
         glob('config/*.yaml')),
        # Installation des scripts
        (os.path.join('share', package_name, 'scripts'), 
         glob('scripts/*.py')),
    ],
    install_requires=['setuptools'],
    zip_safe=True,
    maintainer='Nom Étudiant',
    maintainer_email='etudiant@universite.fr',
    description='Contrôleur PID avec modèle bicyclette pour turtlesim',
    license='MIT',
    tests_require=['pytest'],
    entry_points={
        'console_scripts': [
            # Node principal de contrôle
            'control_node = ros2_pid_turtle.control_node:main',
            # Générateur de trajectoires
            'trajectory_generator = ros2_pid_turtle.trajectory_generator:main',
            # Script d'analyse
            'analyze_performance = ros2_pid_turtle.analyze_performance:main',
        ],
    },
)'''

# Écriture du fichier
with open('~/ros2_ws/src/ros2_pid_turtle/setup.py', 'w') as f:
    f.write(setup_py_content)

print("Fichier setup.py créé avec succès!")



## 3. Modélisation : Modèle Cinématique Bicyclette

### 3.1 Principe et Variables d'État

Le modèle cinématique bicyclette est une représentation simplifiée d'un véhicule à quatre roues où les roues avant et arrière sont regroupées respectivement sur les essieux avant et arrière.

#### Variables d'État du Système

| Variable | Symbole | Unité | Description |
|:---------|:--------|:------|:------------|
| Position X | `x` | m | Coordonnée X dans le repère global |
| Position Y | `y` | m | Coordonnée Y dans le repère global |
| Orientation | `θ` (theta) | rad | Angle par rapport à l'axe X global |
| Vitesse linéaire | `v` | m/s | Vitesse du centre de gravité |
| Angle de braquage | `δ` (delta) | rad | Angle des roues avant |

#### Variables de Commande

| Commande | Symbole | Unité | Description |
|:---------|:--------|:------|:------------|
| Vitesse désirée | `v_cmd` | m/s | Consigne de vitesse linéaire |
| Vitesse angulaire | `ω` (omega) | rad/s | Vitesse de rotation désirée |

### 3.2 Équations Cinématiques

Les équations du modèle bicyclette en temps continu sont:


In [1]:

# Équations mathématiques du modèle bicyclette

# État du système: [x, y, θ, v]
# Commandes: [v_cmd, δ] ou [v_cmd, ω]

# Équations différentielles:
# dx/dt = v * cos(θ)
# dy/dt = v * sin(θ) 
# dθ/dt = (v/L) * tan(δ) = ω
# dv/dt = (v_cmd - v) / τ

# Où:
# L = empattement du véhicule (distance entre essieux)
# τ = constante de temps de la dynamique de vitesse
# δ = angle de braquage des roues avant
# ω = vitesse angulaire (dθ/dt)

# Relations importantes:
# tan(δ) = L * ω / v
# δ = arctan(L * ω / v)

# Pour turtlesim (approximation):
# L ≈ 1.0 (empattement normalisé)
# τ ≈ 0.1 (réponse rapide en vitesse)

print("Équations du modèle bicyclette définies")


Équations du modèle bicyclette définies



### 3.3 Correspondance avec ROS2 geometry_msgs/Twist

Le message `geometry_msgs/msg/Twist` est utilisé pour commander la vitesse des robots dans ROS2:

#### Structure du message Twist

| Champ ROS2 | Type | Utilisation Bicyclette | Description |
|:-----------|:-----|:----------------------|:------------|
| `linear.x` | float64 | `v` (vitesse linéaire) | Vitesse avant/arrière |
| `linear.y` | float64 | `0` (non utilisé) | Vitesse latérale |
| `linear.z` | float64 | `0` (non utilisé) | Vitesse verticale |
| `angular.x` | float64 | `0` (non utilisé) | Vitesse de roulis |
| `angular.y` | float64 | `0` (non utilisé) | Vitesse de tangage |
| `angular.z` | float64 | `ω` (vitesse angulaire) | Vitesse de lacet |

#### Message Pose de turtlesim

Le message `turtlesim/msg/Pose` fournit l'état actuel:

| Champ | Type | Description |
|:------|:-----|:------------|
| `x` | float32 | Position X (0-11.08) |
| `y` | float32 | Position Y (0-11.08) |
| `theta` | float32 | Orientation (-π à π) |
| `linear_velocity` | float32 | Vitesse linéaire actuelle |
| `angular_velocity` | float32 | Vitesse angulaire actuelle |



## 4. Implémentation Python

### 4.1 Classe PID Controller


In [ ]:

# Fichier: ros2_pid_turtle/pid_controller.py

pid_controller_content = '''"""
Contrôleur PID simple pour le projet turtlesim.
Basé sur la librairie simple-pid avec adaptations.
"""

import time
from typing import Optional, Tuple

class PIDController:
    """
    Contrôleur PID simple pour le contrôle de position et d'orientation.
    """

    def __init__(self, 
                 kp: float = 1.0, 
                 ki: float = 0.0, 
                 kd: float = 0.0,
                 setpoint: float = 0.0,
                 output_limits: Optional[Tuple[float, float]] = None,
                 sample_time: float = 0.01):
        """
        Initialise le contrôleur PID.

        Args:
            kp: Gain proportionnel
            ki: Gain intégral  
            kd: Gain dérivé
            setpoint: Consigne à atteindre
            output_limits: Limites de sortie (min, max)
            sample_time: Période d'échantillonnage en secondes
        """
        self.kp = kp
        self.ki = ki
        self.kd = kd
        self.setpoint = setpoint
        self.output_limits = output_limits
        self.sample_time = sample_time

        # Variables internes
        self._last_time = None
        self._last_error = None
        self._integral = 0.0
        self._last_output = None

    def __call__(self, input_value: float, dt: Optional[float] = None) -> float:
        """
        Calcule la sortie PID pour l'entrée donnée.

        Args:
            input_value: Valeur actuelle du système
            dt: Pas de temps (optionnel)

        Returns:
            Sortie du contrôleur PID
        """
        current_time = time.time()

        # Calcul de l'erreur
        error = self.setpoint - input_value

        # Gestion du temps
        if dt is None:
            if self._last_time is None:
                dt = self.sample_time
            else:
                dt = current_time - self._last_time

        # Éviter la division par zéro
        if dt <= 0.0:
            dt = self.sample_time

        # Terme proportionnel
        proportional = self.kp * error

        # Terme intégral
        self._integral += error * dt
        integral = self.ki * self._integral

        # Terme dérivé
        if self._last_error is not None:
            derivative = self.kd * (error - self._last_error) / dt
        else:
            derivative = 0.0

        # Sortie PID
        output = proportional + integral + derivative

        # Application des limites
        if self.output_limits is not None:
            min_output, max_output = self.output_limits
            if output > max_output:
                output = max_output
                # Anti-windup: limiter l'intégrale
                self._integral -= (output - max_output) / (self.ki + 1e-6)
            elif output < min_output:
                output = min_output
                # Anti-windup: limiter l'intégrale
                self._integral -= (output - min_output) / (self.ki + 1e-6)

        # Sauvegarde pour la prochaine itération
        self._last_error = error
        self._last_time = current_time
        self._last_output = output

        return output

    def reset(self):
        """Reset tous les termes internes du PID."""
        self._last_time = None
        self._last_error = None
        self._integral = 0.0
        self._last_output = None

    def set_tunings(self, kp: float, ki: float, kd: float):
        """Modifie les gains du PID."""
        self.kp = kp
        self.ki = ki
        self.kd = kd

    @property
    def components(self) -> Tuple[float, float, float]:
        """Retourne les composantes P, I, D du dernier calcul."""
        if self._last_error is None:
            return 0.0, 0.0, 0.0

        p = self.kp * (self.setpoint - self._last_error)
        i = self.ki * self._integral
        d = self.kd * self._last_error if self._last_error else 0.0

        return p, i, d
'''

# Écriture du fichier
import os
os.makedirs('~/ros2_ws/src/ros2_pid_turtle/ros2_pid_turtle', exist_ok=True)

with open('~/ros2_ws/src/ros2_pid_turtle/ros2_pid_turtle/pid_controller.py', 'w') as f:
    f.write(pid_controller_content)

print("Classe PIDController créée avec succès!")


In [ ]:

# Fichier: ros2_pid_turtle/bicycle_model.py

bicycle_model_content = '''"""
Modèle cinématique bicyclette pour le contrôle de turtlesim.
"""

import math
from typing import Tuple, NamedTuple
from geometry_msgs.msg import Twist

class BicycleState(NamedTuple):
    """État du modèle bicyclette."""
    x: float      # Position X
    y: float      # Position Y  
    theta: float  # Orientation (rad)
    v: float      # Vitesse linéaire

class BicycleCommand(NamedTuple):
    """Commande pour le modèle bicyclette."""
    v_cmd: float    # Vitesse linéaire désirée
    omega: float    # Vitesse angulaire désirée

class BicycleModel:
    """
    Modèle cinématique bicyclette pour turtlesim.
    """

    def __init__(self, 
                 wheelbase: float = 1.0,
                 max_steering_angle: float = math.pi/3,
                 max_velocity: float = 5.0,
                 max_angular_velocity: float = 2.0):
        """
        Initialise le modèle bicyclette.

        Args:
            wheelbase: Empattement du véhicule (L)
            max_steering_angle: Angle de braquage maximum
            max_velocity: Vitesse linéaire maximum
            max_angular_velocity: Vitesse angulaire maximum
        """
        self.wheelbase = wheelbase
        self.max_steering_angle = max_steering_angle
        self.max_velocity = max_velocity
        self.max_angular_velocity = max_angular_velocity

    def compute_steering_angle(self, v: float, omega: float) -> float:
        """
        Calcule l'angle de braquage à partir de v et omega.

        Relation: omega = (v/L) * tan(delta)
        Donc: delta = arctan(L * omega / v)

        Args:
            v: Vitesse linéaire
            omega: Vitesse angulaire

        Returns:
            Angle de braquage (rad)
        """
        if abs(v) < 1e-3:  # Éviter la division par zéro
            return 0.0

        delta = math.atan(self.wheelbase * omega / v)

        # Limiter l'angle de braquage
        delta = max(-self.max_steering_angle, 
                   min(self.max_steering_angle, delta))

        return delta

    def compute_angular_velocity(self, v: float, delta: float) -> float:
        """
        Calcule la vitesse angulaire à partir de v et delta.

        Args:
            v: Vitesse linéaire
            delta: Angle de braquage

        Returns:
            Vitesse angulaire (rad/s)
        """
        omega = (v / self.wheelbase) * math.tan(delta)

        # Limiter la vitesse angulaire
        omega = max(-self.max_angular_velocity,
                   min(self.max_angular_velocity, omega))

        return omega

    def state_update(self, 
                    state: BicycleState, 
                    command: BicycleCommand, 
                    dt: float) -> BicycleState:
        """
        Met à jour l'état du modèle bicyclette.

        Args:
            state: État actuel
            command: Commande à appliquer
            dt: Pas de temps

        Returns:
            Nouvel état
        """
        # Limiter les commandes
        v_cmd = max(-self.max_velocity, 
                   min(self.max_velocity, command.v_cmd))
        omega = max(-self.max_angular_velocity,
                   min(self.max_angular_velocity, command.omega))

        # Dynamique simple de vitesse (filtre du premier ordre)
        tau = 0.1  # Constante de temps
        v_new = state.v + (v_cmd - state.v) * dt / tau

        # Cinématique bicyclette
        x_new = state.x + v_new * math.cos(state.theta) * dt
        y_new = state.y + v_new * math.sin(state.theta) * dt
        theta_new = state.theta + omega * dt

        # Normaliser l'angle
        theta_new = math.atan2(math.sin(theta_new), math.cos(theta_new))

        return BicycleState(x_new, y_new, theta_new, v_new)

    def command_to_twist(self, command: BicycleCommand) -> Twist:
        """
        Convertit une commande bicyclette en message Twist ROS2.

        Args:
            command: Commande bicyclette

        Returns:
            Message Twist pour ROS2
        """
        twist = Twist()

        # Limiter les commandes
        twist.linear.x = max(-self.max_velocity,
                           min(self.max_velocity, command.v_cmd))
        twist.angular.z = max(-self.max_angular_velocity,
                            min(self.max_angular_velocity, command.omega))

        return twist

    def validate_command(self, command: BicycleCommand) -> bool:
        """
        Valide qu'une commande respecte les contraintes physiques.

        Args:
            command: Commande à valider

        Returns:
            True si la commande est valide
        """
        if abs(command.v_cmd) > self.max_velocity:
            return False

        if abs(command.omega) > self.max_angular_velocity:
            return False

        # Vérifier la cohérence cinématique
        if abs(command.v_cmd) > 1e-3:
            required_delta = abs(self.compute_steering_angle(
                command.v_cmd, command.omega))
            if required_delta > self.max_steering_angle:
                return False

        return True
'''

with open('~/ros2_ws/src/ros2_pid_turtle/ros2_pid_turtle/bicycle_model.py', 'w') as f:
    f.write(bicycle_model_content)

print("Classe BicycleModel créée avec succès!")



### 4.2 Node Principal de Contrôle

Le node principal intègre le PID et le modèle bicyclette pour contrôler la tortue.


In [ ]:

# Fichier: ros2_pid_turtle/control_node.py

control_node_content = '''"""
Node principal de contrôle PID pour turtlesim avec modèle bicyclette.
"""

import math
import time
from typing import List, Tuple

import rclpy
from rclpy.node import Node
from rclpy.parameter import Parameter
from geometry_msgs.msg import Twist
from turtlesim.msg import Pose
from std_msgs.msg import Float64MultiArray

from .pid_controller import PIDController
from .bicycle_model import BicycleModel, BicycleState, BicycleCommand

class TurtlePIDController(Node):
    """
    Node de contrôle PID pour turtlesim utilisant un modèle bicyclette.
    """

    def __init__(self):
        super().__init__('turtle_pid_controller')

        # Déclaration des paramètres
        self.declare_parameters(
            namespace='',
            parameters=[
                ('pid_linear.kp', 2.0),
                ('pid_linear.ki', 0.0),
                ('pid_linear.kd', 0.1),
                ('pid_angular.kp', 4.0),
                ('pid_angular.ki', 0.0),
                ('pid_angular.kd', 0.2),
                ('target_x', 9.0),
                ('target_y', 9.0),
                ('target_theta', 0.0),
                ('control_frequency', 50.0),
                ('tolerance_position', 0.1),
                ('tolerance_angle', 0.05),
                ('bicycle.wheelbase', 1.0),
                ('bicycle.max_velocity', 3.0),
                ('bicycle.max_angular_velocity', 2.0)
            ]
        )

        # Récupération des paramètres
        self.target_x = self.get_parameter('target_x').value
        self.target_y = self.get_parameter('target_y').value
        self.target_theta = self.get_parameter('target_theta').value
        self.control_freq = self.get_parameter('control_frequency').value
        self.tolerance_pos = self.get_parameter('tolerance_position').value
        self.tolerance_angle = self.get_parameter('tolerance_angle').value

        # Initialisation du modèle bicyclette
        self.bicycle_model = BicycleModel(
            wheelbase=self.get_parameter('bicycle.wheelbase').value,
            max_velocity=self.get_parameter('bicycle.max_velocity').value,
            max_angular_velocity=self.get_parameter('bicycle.max_angular_velocity').value
        )

        # Initialisation des contrôleurs PID
        self.pid_distance = PIDController(
            kp=self.get_parameter('pid_linear.kp').value,
            ki=self.get_parameter('pid_linear.ki').value,
            kd=self.get_parameter('pid_linear.kd').value,
            output_limits=(0.0, self.bicycle_model.max_velocity)
        )

        self.pid_heading = PIDController(
            kp=self.get_parameter('pid_angular.kp').value,
            ki=self.get_parameter('pid_angular.ki').value,
            kd=self.get_parameter('pid_angular.kd').value,
            output_limits=(-self.bicycle_model.max_angular_velocity,
                          self.bicycle_model.max_angular_velocity)
        )

        # Variables d'état
        self.current_pose = None
        self.target_reached = False
        self.control_active = True

        # Historique pour l'analyse
        self.pose_history = []
        self.command_history = []
        self.error_history = []
        self.time_history = []
        self.start_time = time.time()

        # Publishers et Subscribers
        self.cmd_vel_pub = self.create_publisher(
            Twist, '/turtle1/cmd_vel', 10)

        self.pose_sub = self.create_subscription(
            Pose, '/turtle1/pose', self.pose_callback, 10)

        # Publisher pour les données d'analyse
        self.data_pub = self.create_publisher(
            Float64MultiArray, '/control_data', 10)

        # Timer pour le contrôle
        self.control_timer = self.create_timer(
            1.0 / self.control_freq, self.control_callback)

        # Timer pour la publication des données
        self.data_timer = self.create_timer(
            0.1, self.publish_data)

        self.get_logger().info(f'Contrôleur PID initialisé')
        self.get_logger().info(f'Cible: x={self.target_x:.2f}, y={self.target_y:.2f}, θ={self.target_theta:.2f}')

    def pose_callback(self, msg: Pose):
        """Callback pour la réception de la pose de la tortue."""
        self.current_pose = msg

        # Enregistrer dans l'historique
        current_time = time.time() - self.start_time
        self.time_history.append(current_time)
        self.pose_history.append((msg.x, msg.y, msg.theta))

    def control_callback(self):
        """Callback principal de contrôle."""
        if self.current_pose is None:
            return

        if self.target_reached:
            # Arrêter la tortue si la cible est atteinte
            self.publish_stop_command()
            return

        # Calcul des erreurs
        error_x = self.target_x - self.current_pose.x
        error_y = self.target_y - self.current_pose.y
        distance_error = math.sqrt(error_x**2 + error_y**2)

        # Angle vers la cible
        angle_to_target = math.atan2(error_y, error_x)
        angle_error = self.normalize_angle(angle_to_target - self.current_pose.theta)

        # Vérifier si la cible est atteinte
        if (distance_error < self.tolerance_pos and 
            abs(angle_error) < self.tolerance_angle):
            self.target_reached = True
            self.get_logger().info('Cible atteinte!')
            self.publish_stop_command()
            return

        # Mise à jour des consignes PID
        self.pid_distance.setpoint = 0.0  # On veut réduire la distance à 0
        self.pid_heading.setpoint = 0.0   # On veut réduire l'erreur d'angle à 0

        # Calcul des commandes PID
        if distance_error > self.tolerance_pos:
            # Priorité à l'orientation si on est loin
            if abs(angle_error) > math.pi/4:  # 45 degrés
                velocity_cmd = 0.5  # Vitesse réduite pour tourner
                angular_cmd = self.pid_heading(-angle_error)
            else:
                # Avancer vers la cible
                velocity_cmd = self.pid_distance(-distance_error)
                angular_cmd = self.pid_heading(-angle_error)
        else:
            # Ajustement final de l'orientation
            final_angle_error = self.normalize_angle(self.target_theta - self.current_pose.theta)
            velocity_cmd = 0.0
            angular_cmd = self.pid_heading(-final_angle_error)

        # Créer la commande bicyclette
        bicycle_cmd = BicycleCommand(velocity_cmd, angular_cmd)

        # Valider et convertir en Twist
        if self.bicycle_model.validate_command(bicycle_cmd):
            twist_msg = self.bicycle_model.command_to_twist(bicycle_cmd)
            self.cmd_vel_pub.publish(twist_msg)

            # Enregistrer dans l'historique
            self.command_history.append((velocity_cmd, angular_cmd))
            self.error_history.append((distance_error, angle_error))
        else:
            self.get_logger().warn('Commande invalide générée par le PID')
            self.publish_stop_command()

    def publish_stop_command(self):
        """Publie une commande d'arrêt."""
        stop_msg = Twist()
        self.cmd_vel_pub.publish(stop_msg)

    def publish_data(self):
        """Publie les données pour l'analyse."""
        if self.current_pose is None:
            return

        # Préparer les données
        data_msg = Float64MultiArray()
        current_time = time.time() - self.start_time

        error_x = self.target_x - self.current_pose.x
        error_y = self.target_y - self.current_pose.y
        distance_error = math.sqrt(error_x**2 + error_y**2)

        angle_to_target = math.atan2(error_y, error_x)
        angle_error = self.normalize_angle(angle_to_target - self.current_pose.theta)

        # Format: [time, x, y, theta, target_x, target_y, target_theta, 
        #          distance_error, angle_error, cmd_vel, cmd_omega]
        data_msg.data = [
            current_time,
            float(self.current_pose.x),
            float(self.current_pose.y), 
            float(self.current_pose.theta),
            self.target_x,
            self.target_y,
            self.target_theta,
            distance_error,
            angle_error,
            0.0,  # cmd_vel (sera mis à jour)
            0.0   # cmd_omega (sera mis à jour)
        ]

        if self.command_history:
            data_msg.data[-2] = self.command_history[-1][0]  # cmd_vel
            data_msg.data[-1] = self.command_history[-1][1]  # cmd_omega

        self.data_pub.publish(data_msg)

    def normalize_angle(self, angle: float) -> float:
        """Normalise un angle entre -π et π."""
        return math.atan2(math.sin(angle), math.cos(angle))

    def set_target(self, x: float, y: float, theta: float = 0.0):
        """Change la cible du contrôleur."""
        self.target_x = x
        self.target_y = y
        self.target_theta = theta
        self.target_reached = False

        # Reset des PID
        self.pid_distance.reset()
        self.pid_heading.reset()

        self.get_logger().info(f'Nouvelle cible: x={x:.2f}, y={y:.2f}, θ={theta:.2f}')

    def get_performance_data(self) -> dict:
        """Retourne les données de performance pour l'analyse."""
        return {
            'time_history': self.time_history.copy(),
            'pose_history': self.pose_history.copy(),
            'command_history': self.command_history.copy(),
            'error_history': self.error_history.copy(),
            'target': (self.target_x, self.target_y, self.target_theta),
            'target_reached': self.target_reached
        }

def main(args=None):
    """Point d'entrée principal."""
    rclpy.init(args=args)

    controller = TurtlePIDController()

    try:
        rclpy.spin(controller)
    except KeyboardInterrupt:
        controller.get_logger().info('Arrêt du contrôleur')
    finally:
        # Arrêter la tortue
        controller.publish_stop_command()
        controller.destroy_node()
        rclpy.shutdown()

if __name__ == '__main__':
    main()
'''

with open('~/ros2_ws/src/ros2_pid_turtle/ros2_pid_turtle/control_node.py', 'w') as f:
    f.write(control_node_content)

print("Node principal de contrôle créé avec succès!")



### 4.3 Générateur de Trajectoires

Ce module permet de générer différents types de trajectoires de référence.


In [ ]:

# Fichier: ros2_pid_turtle/trajectory_generator.py

trajectory_content = '''"""
Générateur de trajectoires pour le contrôle de turtlesim.
"""

import math
import time
from typing import List, Tuple, Generator

import rclpy
from rclpy.node import Node
from geometry_msgs.msg import PoseStamped
from std_msgs.msg import Header

class TrajectoryGenerator(Node):
    """
    Générateur de trajectoires de référence pour turtlesim.
    """

    def __init__(self):
        super().__init__('trajectory_generator')

        # Paramètres
        self.declare_parameters(
            namespace='',
            parameters=[
                ('trajectory_type', 'circle'),  # circle, square, figure8, waypoints
                ('center_x', 5.5),
                ('center_y', 5.5),
                ('radius', 2.0),
                ('period', 20.0),  # Période en secondes
                ('publish_rate', 10.0),  # Hz
                ('square_size', 3.0),
            ]
        )

        # Récupération des paramètres
        self.traj_type = self.get_parameter('trajectory_type').value
        self.center_x = self.get_parameter('center_x').value
        self.center_y = self.get_parameter('center_y').value
        self.radius = self.get_parameter('radius').value
        self.period = self.get_parameter('period').value
        self.publish_rate = self.get_parameter('publish_rate').value
        self.square_size = self.get_parameter('square_size').value

        # Publisher
        self.ref_pub = self.create_publisher(
            PoseStamped, '/reference_pose', 10)

        # Timer
        self.timer = self.create_timer(
            1.0 / self.publish_rate, self.publish_reference)

        # Variables
        self.start_time = time.time()
        self.waypoints = self.generate_waypoints()
        self.current_waypoint_idx = 0

        self.get_logger().info(f'Générateur de trajectoire initialisé: {self.traj_type}')

    def generate_waypoints(self) -> List[Tuple[float, float, float]]:
        """Génère les points de passage selon le type de trajectoire."""
        waypoints = []

        if self.traj_type == 'circle':
            # Trajectoire circulaire
            n_points = int(self.publish_rate * self.period)
            for i in range(n_points):
                t = 2 * math.pi * i / n_points
                x = self.center_x + self.radius * math.cos(t)
                y = self.center_y + self.radius * math.sin(t)
                theta = t + math.pi/2  # Tangent à la trajectoire
                waypoints.append((x, y, theta))

        elif self.traj_type == 'square':
            # Trajectoire carrée
            half_size = self.square_size / 2
            corners = [
                (self.center_x - half_size, self.center_y - half_size, 0.0),
                (self.center_x + half_size, self.center_y - half_size, math.pi/2),
                (self.center_x + half_size, self.center_y + half_size, math.pi),
                (self.center_x - half_size, self.center_y + half_size, -math.pi/2),
            ]

            # Interpolation entre les coins
            points_per_side = int(self.publish_rate * self.period / 4)
            for i in range(4):
                start = corners[i]
                end = corners[(i + 1) % 4]

                for j in range(points_per_side):
                    alpha = j / points_per_side
                    x = start[0] + alpha * (end[0] - start[0])
                    y = start[1] + alpha * (end[1] - start[1])
                    theta = end[2]  # Orientation vers le prochain coin
                    waypoints.append((x, y, theta))

        elif self.traj_type == 'figure8':
            # Trajectoire en forme de 8
            n_points = int(self.publish_rate * self.period)
            for i in range(n_points):
                t = 2 * math.pi * i / n_points
                # Paramètres pour le 8
                x = self.center_x + self.radius * math.sin(t)
                y = self.center_y + self.radius * math.sin(t) * math.cos(t)
                # Calcul de l'orientation tangente
                dx_dt = self.radius * math.cos(t)
                dy_dt = self.radius * (math.cos(2*t))
                theta = math.atan2(dy_dt, dx_dt)
                waypoints.append((x, y, theta))

        elif self.traj_type == 'waypoints':
            # Points de passage prédéfinis
            predefined_waypoints = [
                (2.0, 2.0, 0.0),
                (9.0, 2.0, math.pi/2),
                (9.0, 9.0, math.pi),
                (2.0, 9.0, -math.pi/2),
                (5.5, 5.5, 0.0),
            ]

            # Interpolation entre les waypoints
            points_per_segment = int(self.publish_rate * self.period / len(predefined_waypoints))
            for i in range(len(predefined_waypoints)):
                start = predefined_waypoints[i]
                end = predefined_waypoints[(i + 1) % len(predefined_waypoints)]

                for j in range(points_per_segment):
                    alpha = j / points_per_segment
                    x = start[0] + alpha * (end[0] - start[0])
                    y = start[1] + alpha * (end[1] - start[1])
                    theta = math.atan2(end[1] - start[1], end[0] - start[0])
                    waypoints.append((x, y, theta))

        else:
            # Point fixe par défaut
            waypoints = [(self.center_x, self.center_y, 0.0)]

        return waypoints

    def publish_reference(self):
        """Publie la pose de référence courante."""
        if not self.waypoints:
            return

        # Pose courante dans la trajectoire
        current_pose = self.waypoints[self.current_waypoint_idx]

        # Création du message
        pose_msg = PoseStamped()
        pose_msg.header = Header()
        pose_msg.header.stamp = self.get_clock().now().to_msg()
        pose_msg.header.frame_id = 'map'

        pose_msg.pose.position.x = current_pose[0]
        pose_msg.pose.position.y = current_pose[1]
        pose_msg.pose.position.z = 0.0

        # Conversion angle en quaternion (rotation autour de z)
        theta = current_pose[2]
        pose_msg.pose.orientation.x = 0.0
        pose_msg.pose.orientation.y = 0.0
        pose_msg.pose.orientation.z = math.sin(theta / 2.0)
        pose_msg.pose.orientation.w = math.cos(theta / 2.0)

        self.ref_pub.publish(pose_msg)

        # Passage au waypoint suivant
        self.current_waypoint_idx = (self.current_waypoint_idx + 1) % len(self.waypoints)

    def get_trajectory_info(self) -> dict:
        """Retourne les informations sur la trajectoire."""
        return {
            'type': self.traj_type,
            'waypoints': self.waypoints,
            'total_points': len(self.waypoints),
            'period': self.period,
            'current_index': self.current_waypoint_idx
        }

def main(args=None):
    """Point d'entrée principal."""
    rclpy.init(args=args)

    trajectory_generator = TrajectoryGenerator()

    try:
        rclpy.spin(trajectory_generator)
    except KeyboardInterrupt:
        trajectory_generator.get_logger().info('Arrêt du générateur de trajectoire')
    finally:
        trajectory_generator.destroy_node()
        rclpy.shutdown()

if __name__ == '__main__':
    main()
'''

with open('~/ros2_ws/src/ros2_pid_turtle/ros2_pid_turtle/trajectory_generator.py', 'w') as f:
    f.write(trajectory_content)

print("Générateur de trajectoires créé avec succès!")



## 5. Fichiers de Configuration et Launch

### 5.1 Configuration des Paramètres PID


In [ ]:

# Fichier: config/pid_params.yaml

pid_params_content = '''# Paramètres du contrôleur PID pour turtlesim
# Configuration optimisée pour le simulateur

# Contrôleur PID pour la distance (vitesse linéaire)
pid_linear:
  kp: 2.0      # Gain proportionnel - réactivité à l'erreur de distance
  ki: 0.1      # Gain intégral - élimination erreur statique
  kd: 0.2      # Gain dérivé - atténuation des oscillations

# Contrôleur PID pour l'orientation (vitesse angulaire)  
pid_angular:
  kp: 4.0      # Gain proportionnel - réactivité à l'erreur d'angle
  ki: 0.0      # Gain intégral - généralement 0 pour éviter windup
  kd: 0.3      # Gain dérivé - stabilisation des rotations

# Position cible par défaut
target_x: 9.0        # Position X cible (0-11.08)
target_y: 9.0        # Position Y cible (0-11.08) 
target_theta: 0.0    # Orientation cible en radians

# Paramètres de contrôle
control_frequency: 50.0      # Fréquence du contrôleur en Hz
tolerance_position: 0.1      # Tolérance de position en mètres
tolerance_angle: 0.05        # Tolérance d'angle en radians

# Paramètres du modèle bicyclette
bicycle:
  wheelbase: 1.0                    # Empattement du véhicule
  max_velocity: 3.0                 # Vitesse linéaire max (m/s)
  max_angular_velocity: 2.0         # Vitesse angulaire max (rad/s)
  max_steering_angle: 1.047         # Angle braquage max (60°)

# Paramètres de sécurité
safety:
  emergency_stop_distance: 0.5      # Distance d'arrêt d'urgence
  max_error_threshold: 5.0          # Erreur max avant reset
  timeout_seconds: 30.0             # Timeout de la mission
'''

# Créer le dossier config et écrire le fichier
import os
os.makedirs('~/ros2_ws/src/ros2_pid_turtle/config', exist_ok=True)

with open('~/ros2_ws/src/ros2_pid_turtle/config/pid_params.yaml', 'w') as f:
    f.write(pid_params_content)

print("Fichier de configuration PID créé avec succès!")


In [ ]:

# Fichier: config/trajectory_params.yaml

trajectory_params_content = '''# Paramètres des trajectoires de référence

# Type de trajectoire: circle, square, figure8, waypoints, point
trajectory_type: "circle"

# Paramètres généraux
center_x: 5.5        # Centre X de la trajectoire
center_y: 5.5        # Centre Y de la trajectoire
publish_rate: 10.0   # Fréquence de publication (Hz)

# Trajectoire circulaire
circle:
  radius: 2.0        # Rayon du cercle
  period: 20.0       # Période complète en secondes
  clockwise: false   # Sens de rotation

# Trajectoire carrée
square:
  size: 3.0          # Taille du côté
  period: 16.0       # Période complète en secondes
  corner_pause: 1.0  # Pause aux coins en secondes

# Trajectoire en 8
figure8:
  width: 4.0         # Largeur du 8
  height: 2.0        # Hauteur du 8  
  period: 25.0       # Période complète en secondes

# Points de passage prédéfinis
waypoints:
  points:
    - [2.0, 2.0, 0.0]      # [x, y, theta]
    - [9.0, 2.0, 1.57]     # π/2
    - [9.0, 9.0, 3.14]     # π  
    - [2.0, 9.0, -1.57]    # -π/2
    - [5.5, 5.5, 0.0]      # Centre
  dwell_time: 2.0          # Temps d'arrêt à chaque point

# Point fixe
point:
  x: 5.5
  y: 5.5
  theta: 0.0
'''

with open('~/ros2_ws/src/ros2_pid_turtle/config/trajectory_params.yaml', 'w') as f:
    f.write(trajectory_params_content)

print("Fichier de configuration trajectoires créé avec succès!")



### 5.2 Fichiers de Lancement (Launch Files)

Les fichiers launch permettent de démarrer plusieurs nodes avec leurs configurations.


In [ ]:

# Fichier: launch/pid_control.launch.py

launch_content = '''"""
Fichier launch principal pour le contrôle PID de turtlesim.
Démarre turtlesim et le contrôleur PID avec les paramètres.
"""

import os
from ament_index_python.packages import get_package_share_directory
from launch import LaunchDescription
from launch.actions import DeclareLaunchArgument, IncludeLaunchDescription
from launch.conditions import IfCondition
from launch.launch_description_sources import PythonLaunchDescriptionSource
from launch.substitutions import LaunchConfiguration, PathJoinSubstitution
from launch_ros.actions import Node
from launch_ros.substitutions import FindPackageShare

def generate_launch_description():
    """Génère la description du launch."""

    # Chemin vers le package
    pkg_dir = get_package_share_directory('ros2_pid_turtle')

    # Arguments de lancement
    use_sim_time_arg = DeclareLaunchArgument(
        'use_sim_time',
        default_value='false',
        description='Utiliser le temps de simulation'
    )

    config_file_arg = DeclareLaunchArgument(
        'config_file',
        default_value=os.path.join(pkg_dir, 'config', 'pid_params.yaml'),
        description='Fichier de configuration des paramètres'
    )

    target_x_arg = DeclareLaunchArgument(
        'target_x',
        default_value='9.0',
        description='Position X cible'
    )

    target_y_arg = DeclareLaunchArgument(
        'target_y', 
        default_value='9.0',
        description='Position Y cible'
    )

    target_theta_arg = DeclareLaunchArgument(
        'target_theta',
        default_value='0.0', 
        description='Orientation cible (radians)'
    )

    enable_logging_arg = DeclareLaunchArgument(
        'enable_logging',
        default_value='true',
        description='Activer l\'enregistrement des données'
    )

    # Node turtlesim
    turtlesim_node = Node(
        package='turtlesim',
        executable='turtlesim_node',
        name='turtlesim',
        output='screen',
        parameters=[{
            'use_sim_time': LaunchConfiguration('use_sim_time'),
            'background_r': 69,   # Couleur de fond personnalisée
            'background_g': 86, 
            'background_b': 255
        }]
    )

    # Node contrôleur PID  
    pid_controller_node = Node(
        package='ros2_pid_turtle',
        executable='control_node',
        name='pid_controller',
        output='screen',
        parameters=[
            LaunchConfiguration('config_file'),
            {
                'use_sim_time': LaunchConfiguration('use_sim_time'),
                'target_x': LaunchConfiguration('target_x'),
                'target_y': LaunchConfiguration('target_y'),
                'target_theta': LaunchConfiguration('target_theta'),
            }
        ]
    )

    # Node d\'enregistrement des données (conditionnel)
    data_recorder_node = Node(
        package='ros2_pid_turtle',
        executable='data_recorder',
        name='data_recorder',
        output='screen',
        condition=IfCondition(LaunchConfiguration('enable_logging')),
        parameters=[{
            'use_sim_time': LaunchConfiguration('use_sim_time'),
            'output_file': '/tmp/turtle_control_data.csv'
        }]
    )

    return LaunchDescription([
        # Arguments
        use_sim_time_arg,
        config_file_arg,
        target_x_arg,
        target_y_arg,
        target_theta_arg,
        enable_logging_arg,

        # Nodes
        turtlesim_node,
        pid_controller_node,
        data_recorder_node,
    ])
'''

# Créer le dossier launch et écrire le fichier
os.makedirs('~/ros2_ws/src/ros2_pid_turtle/launch', exist_ok=True)

with open('~/ros2_ws/src/ros2_pid_turtle/launch/pid_control.launch.py', 'w') as f:
    f.write(launch_content)

print("Fichier launch principal créé avec succès!")


In [ ]:

# Fichier: launch/analysis.launch.py

analysis_launch_content = '''"""
Fichier launch pour l\'analyse et la visualisation des performances.
Inclut l\'enregistrement de données et la génération de graphiques.
"""

import os
from ament_index_python.packages import get_package_share_directory
from launch import LaunchDescription
from launch.actions import DeclareLaunchArgument, ExecuteProcess
from launch.substitutions import LaunchConfiguration
from launch_ros.actions import Node

def generate_launch_description():
    """Génère la description du launch pour l\'analyse."""

    pkg_dir = get_package_share_directory('ros2_pid_turtle')

    # Arguments
    bag_file_arg = DeclareLaunchArgument(
        'bag_file',
        default_value='/tmp/turtle_experiment',
        description='Nom du fichier rosbag'
    )

    experiment_duration_arg = DeclareLaunchArgument(
        'experiment_duration',
        default_value='60',
        description='Durée de l\'expérience en secondes'
    )

    trajectory_type_arg = DeclareLaunchArgument(
        'trajectory_type',
        default_value='circle',
        description='Type de trajectoire à suivre'
    )

    # Node turtlesim
    turtlesim_node = Node(
        package='turtlesim',
        executable='turtlesim_node',
        name='turtlesim',
        output='screen'
    )

    # Node contrôleur PID
    pid_controller_node = Node(
        package='ros2_pid_turtle',
        executable='control_node',
        name='pid_controller',
        output='screen',
        parameters=[
            os.path.join(pkg_dir, 'config', 'pid_params.yaml')
        ]
    )

    # Node générateur de trajectoire
    trajectory_node = Node(
        package='ros2_pid_turtle',
        executable='trajectory_generator',
        name='trajectory_generator',
        output='screen',
        parameters=[{
            'trajectory_type': LaunchConfiguration('trajectory_type'),
        }]
    )

    # Enregistrement rosbag
    rosbag_record = ExecuteProcess(
        cmd=[
            'ros2', 'bag', 'record',
            '/turtle1/pose',
            '/turtle1/cmd_vel', 
            '/control_data',
            '/reference_pose',
            '-o', LaunchConfiguration('bag_file')
        ],
        output='screen'
    )

    # Arrêt automatique après la durée spécifiée
    experiment_timer = ExecuteProcess(
        cmd=[
            'bash', '-c',
            f'sleep {LaunchConfiguration("experiment_duration")} && '
            'ros2 lifecycle set /pid_controller shutdown'
        ],
        output='screen'
    )

    return LaunchDescription([
        # Arguments
        bag_file_arg,
        experiment_duration_arg,
        trajectory_type_arg,

        # Nodes
        turtlesim_node,
        pid_controller_node,
        trajectory_node,
        rosbag_record,
        experiment_timer,
    ])
'''

with open('~/ros2_ws/src/ros2_pid_turtle/launch/analysis.launch.py', 'w') as f:
    f.write(analysis_launch_content)

print("Fichier launch pour l'analyse créé avec succès!")



## 6. Commandes ROS2 Complètes Step-by-Step

### 6.1 Séquence Complète de Compilation et Exécution

Voici toutes les commandes nécessaires pour compiler et exécuter le projet:


In [ ]:

# Commandes ROS2 step-by-step - À exécuter dans le terminal

# ===== PHASE 1: PRÉPARATION DE L'ENVIRONNEMENT =====

# 1. Source de l'environnement ROS2 (à faire dans chaque nouveau terminal)
echo "source /opt/ros/dashing/setup.bash" >> ~/.bashrc
source ~/.bashrc

# 2. Aller dans le workspace
cd ~/ros2_ws

# 3. Installer les dépendances
rosdep install --from-paths src --ignore-src -r -y

# ===== PHASE 2: COMPILATION =====

# 4. Build du workspace complet
colcon build --packages-select ros2_pid_turtle

# Alternative: build avec debug pour le développement
# colcon build --packages-select ros2_pid_turtle --cmake-args -DCMAKE_BUILD_TYPE=Debug

# 5. Source de l'environnement local
source install/setup.bash

# ===== PHASE 3: VÉRIFICATION =====

# 6. Vérifier que le package est bien installé
ros2 pkg list | grep ros2_pid_turtle

# 7. Vérifier les executables
ros2 pkg executables ros2_pid_turtle

# 8. Vérifier les launch files
ros2 launch ros2_pid_turtle --show-args pid_control.launch.py

print("✅ Commandes de compilation définies")


In [ ]:

# ===== PHASE 4: EXÉCUTION BASIQUE =====

# Terminal 1: Lancer turtlesim manuellement
ros2 run turtlesim turtlesim_node

# Terminal 2: Lancer le contrôleur PID avec paramètres par défaut  
ros2 run ros2_pid_turtle control_node

# ===== PHASE 5: EXÉCUTION AVEC LAUNCH FILE =====

# Option A: Launch basique (tout en un)
ros2 launch ros2_pid_turtle pid_control.launch.py

# Option B: Launch avec paramètres personnalisés
ros2 launch ros2_pid_turtle pid_control.launch.py target_x:=8.0 target_y:=3.0 target_theta:=1.57

# Option C: Launch pour analyse complète
ros2 launch ros2_pid_turtle analysis.launch.py trajectory_type:=circle experiment_duration:=120

# ===== PHASE 6: MONITORING ET DEBUG =====

# Terminal 3: Visualiser les topics actifs
ros2 topic list

# Monitoring des données
ros2 topic echo /turtle1/pose
ros2 topic echo /turtle1/cmd_vel  
ros2 topic echo /control_data

# Visualiser la fréquence des messages
ros2 topic hz /turtle1/pose
ros2 topic hz /turtle1/cmd_vel

# Informations sur les nodes
ros2 node list
ros2 node info /pid_controller

# Graphe du système  
rqt_graph

print("✅ Commandes d'exécution et monitoring définies")


In [ ]:

# ===== PHASE 7: ENREGISTREMENT DE DONNÉES =====

# Enregistrement rosbag manuel
ros2 bag record /turtle1/pose /turtle1/cmd_vel /control_data -o turtle_experiment_$(date +%Y%m%d_%H%M%S)

# Enregistrement tous les topics
ros2 bag record -a -o full_experiment_$(date +%Y%m%d_%H%M%S)

# Enregistrement avec durée limitée (60 secondes)
timeout 60 ros2 bag record /turtle1/pose /turtle1/cmd_vel -o experiment_60s

# ===== PHASE 8: REPLAY ET ANALYSE =====

# Informations sur un bag
ros2 bag info turtle_experiment_20250828_235900

# Replay des données
ros2 bag play turtle_experiment_20250828_235900

# Replay avec vitesse modifiée
ros2 bag play turtle_experiment_20250828_235900 --rate 0.5

# Export en CSV pour analyse
ros2 bag play turtle_experiment_20250828_235900 &
ros2 topic echo /turtle1/pose --csv > pose_data.csv

# ===== PHASE 9: PARAMÉTRAGE EN TEMPS RÉEL =====

# Lister les paramètres du contrôleur
ros2 param list /pid_controller

# Modifier les gains PID en temps réel
ros2 param set /pid_controller pid_linear.kp 3.0
ros2 param set /pid_controller pid_angular.kd 0.5

# Sauvegarder les paramètres actuels
ros2 param dump /pid_controller > current_params.yaml

# Charger de nouveaux paramètres
ros2 param load /pid_controller new_params.yaml

print("✅ Commandes d'enregistrement et paramétrage définies")


In [ ]:

# ===== PHASE 10: DÉPANNAGE ET TESTS =====

# Vérifier les erreurs de compilation
colcon build --packages-select ros2_pid_turtle --event-handlers console_direct+

# Tests unitaires (si implémentés)
colcon test --packages-select ros2_pid_turtle
colcon test-result --all

# Logs détaillés
ros2 run ros2_pid_turtle control_node --ros-args --log-level DEBUG

# Interface pour modifier la cible interactivement
ros2 service call /set_target geometry_msgs/srv/Pose "position: {x: 7.0, y: 4.0, z: 0.0}"

# Arrêt d'urgence
ros2 topic pub /turtle1/cmd_vel geometry_msgs/msg/Twist "{linear: {x: 0.0}, angular: {z: 0.0}}" --once

# Reset de la simulation
ros2 service call /reset std_srvs/srv/Empty

# Monitoring des ressources
htop  # CPU/RAM usage
iostat 1  # I/O monitoring

# ===== PHASE 11: NETTOYAGE =====

# Arrêter tous les nodes ROS2
pkill -f ros2

# Nettoyer les build files
rm -rf ~/ros2_ws/build/ ~/ros2_ws/install/ ~/ros2_ws/log/

# Re-build complet
cd ~/ros2_ws && colcon build

print("✅ Commandes de dépannage et nettoyage définies")



### 6.2 Tableau Récapitulatif des Commandes Principales

| Phase | Commande | Description | Terminal |
|:------|:---------|:------------|:---------|
| **Setup** | `source /opt/ros/dashing/setup.bash` | Source ROS2 | Tous |
| **Build** | `colcon build --packages-select ros2_pid_turtle` | Compilation | 1 |
| **Source** | `source install/setup.bash` | Source workspace | Tous |
| **Launch** | `ros2 launch ros2_pid_turtle pid_control.launch.py` | Exécution complète | 1 |
| **Monitor** | `ros2 topic echo /turtle1/pose` | Voir position | 2 |
| **Graph** | `rqt_graph` | Visualiser connexions | 3 |
| **Record** | `ros2 bag record /turtle1/pose /turtle1/cmd_vel -o exp` | Enregistrer | 4 |
| **Params** | `ros2 param set /pid_controller pid_linear.kp 2.5` | Modifier PID | 2 |
| **Stop** | `Ctrl+C` puis `pkill -f ros2` | Arrêt complet | Tous |

### 6.3 Workflow de Développement Recommandé

1. **Phase de développement** : Compilation incrémentale + tests manuels
2. **Phase de test** : Launch files + monitoring + paramétrage
3. **Phase d'expérimentation** : Enregistrement rosbag + analyse
4. **Phase de validation** : Tests automatisés + métriques de performance



## 7. Analyse des Performances et Visualisation

### 7.1 Script d'Analyse des Données

Ce script analyse les données enregistrées et génère des métriques de performance.


In [ ]:

# Fichier: scripts/analyze_performance.py

analysis_script_content = '''#!/usr/bin/env python3
"""
Script d\'analyse des performances du contrôleur PID.
Génère des métriques et graphiques à partir des données rosbag.
"""

import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from typing import Dict, List, Tuple
import argparse
import os

class PIDPerformanceAnalyzer:
    """Analyseur de performance pour le contrôleur PID turtlesim."""

    def __init__(self, data_file: str):
        """
        Initialise l\'analyseur avec un fichier de données.

        Args:
            data_file: Chemin vers le fichier CSV des données
        """
        self.data_file = data_file
        self.data = None
        self.metrics = {}

    def load_data(self) -> bool:
        """
        Charge les données depuis le fichier CSV.

        Returns:
            True si le chargement réussit
        """
        try:
            # Format attendu: time,x,y,theta,target_x,target_y,target_theta,dist_error,angle_error,cmd_vel,cmd_omega
            self.data = pd.read_csv(self.data_file)

            # Vérifier les colonnes requises
            required_cols = ['time', 'x', 'y', 'theta', 'target_x', 'target_y', 
                           'dist_error', 'angle_error', 'cmd_vel', 'cmd_omega']

            missing_cols = [col for col in required_cols if col not in self.data.columns]
            if missing_cols:
                print(f"Colonnes manquantes: {missing_cols}")
                return False

            print(f"Données chargées: {len(self.data)} échantillons")
            return True

        except Exception as e:
            print(f"Erreur lors du chargement: {e}")
            return False

    def calculate_metrics(self) -> Dict:
        """
        Calcule les métriques de performance.

        Returns:
            Dictionnaire des métriques
        """
        if self.data is None:
            return {}

        # Erreurs
        dist_errors = self.data['dist_error'].values
        angle_errors = self.data['angle_error'].values

        # Métriques d\'erreur
        self.metrics = {
            # Erreur de position
            'rmse_position': np.sqrt(np.mean(dist_errors**2)),
            'mae_position': np.mean(np.abs(dist_errors)),
            'max_error_position': np.max(dist_errors),
            'final_error_position': dist_errors[-1] if len(dist_errors) > 0 else 0,

            # Erreur d\'orientation
            'rmse_orientation': np.sqrt(np.mean(angle_errors**2)),
            'mae_orientation': np.mean(np.abs(angle_errors)),
            'max_error_orientation': np.max(np.abs(angle_errors)),
            'final_error_orientation': angle_errors[-1] if len(angle_errors) > 0 else 0,

            # Temps de réponse
            'settling_time_pos': self._calculate_settling_time(dist_errors, 0.05),
            'settling_time_angle': self._calculate_settling_time(np.abs(angle_errors), 0.05),

            # Dépassement
            'overshoot_position': self._calculate_overshoot(dist_errors),

            # Commandes
            'mean_cmd_vel': np.mean(self.data['cmd_vel']),
            'max_cmd_vel': np.max(np.abs(self.data['cmd_vel'])),
            'mean_cmd_omega': np.mean(np.abs(self.data['cmd_omega'])),
            'max_cmd_omega': np.max(np.abs(self.data['cmd_omega'])),

            # Efficacité énergétique (approximation)
            'energy_index': np.mean(self.data['cmd_vel']**2 + self.data['cmd_omega']**2),

            # Durée totale
            'total_time': self.data['time'].iloc[-1] - self.data['time'].iloc[0],
            'sample_rate': len(self.data) / (self.data['time'].iloc[-1] - self.data['time'].iloc[0])
        }

        return self.metrics

    def _calculate_settling_time(self, error_signal: np.ndarray, tolerance: float) -> float:
        """Calcule le temps de stabilisation."""
        # Trouver le dernier moment où l\'erreur dépasse la tolérance
        above_tolerance = np.abs(error_signal) > tolerance
        if not any(above_tolerance):
            return 0.0

        last_violation_idx = np.where(above_tolerance)[0][-1]
        settling_time = self.data['time'].iloc[last_violation_idx]
        return settling_time - self.data['time'].iloc[0]

    def _calculate_overshoot(self, error_signal: np.ndarray) -> float:
        """Calcule le dépassement maximal."""
        # Pour un système de poursuite, le dépassement est le pic d\'erreur initial
        if len(error_signal) < 10:
            return 0.0

        initial_error = error_signal[0]
        if initial_error == 0:
            return 0.0

        min_error = np.min(error_signal[:len(error_signal)//2])  # Première moitié
        overshoot = abs(min_error - initial_error) / abs(initial_error) * 100
        return overshoot

    def generate_plots(self, output_dir: str = '.'):
        """
        Génère les graphiques d\'analyse.

        Args:
            output_dir: Répertoire de sortie pour les graphiques
        """
        if self.data is None:
            return

        # Configuration matplotlib
        plt.style.use('seaborn-v0_8')
        fig_size = (15, 12)

        # Figure 1: Trajectoire et erreurs
        fig1, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=fig_size)

        # Trajectoire XY
        ax1.plot(self.data['x'], self.data['y'], 'b-', linewidth=2, label='Trajectoire réelle')
        ax1.plot(self.data['target_x'], self.data['target_y'], 'r--', linewidth=2, label='Cible')
        ax1.plot(self.data['x'].iloc[0], self.data['y'].iloc[0], 'go', markersize=8, label='Début')
        ax1.plot(self.data['x'].iloc[-1], self.data['y'].iloc[-1], 'ro', markersize=8, label='Fin')
        ax1.set_xlabel('X (m)')
        ax1.set_ylabel('Y (m)')
        ax1.set_title('Trajectoire dans le Plan XY')
        ax1.legend()
        ax1.grid(True)
        ax1.axis('equal')

        # Erreur de position
        ax2.plot(self.data['time'], self.data['dist_error'], 'b-', linewidth=2)
        ax2.set_xlabel('Temps (s)')
        ax2.set_ylabel('Erreur de distance (m)')
        ax2.set_title('Erreur de Position')
        ax2.grid(True)
        ax2.axhline(y=0.1, color='r', linestyle='--', alpha=0.7, label='Tolérance')
        ax2.legend()

        # Erreur d\'orientation
        ax3.plot(self.data['time'], np.degrees(self.data['angle_error']), 'g-', linewidth=2)
        ax3.set_xlabel('Temps (s)')
        ax3.set_ylabel('Erreur d\'angle (°)')
        ax3.set_title('Erreur d\'Orientation')
        ax3.grid(True)
        ax3.axhline(y=np.degrees(0.05), color='r', linestyle='--', alpha=0.7, label='Tolérance')
        ax3.legend()

        # Commandes
        ax4_twin = ax4.twinx()
        line1 = ax4.plot(self.data['time'], self.data['cmd_vel'], 'b-', linewidth=2, label='Vitesse linéaire')
        line2 = ax4_twin.plot(self.data['time'], self.data['cmd_omega'], 'r-', linewidth=2, label='Vitesse angulaire')
        ax4.set_xlabel('Temps (s)')
        ax4.set_ylabel('Vitesse linéaire (m/s)', color='b')
        ax4_twin.set_ylabel('Vitesse angulaire (rad/s)', color='r')
        ax4.set_title('Signaux de Commande')
        ax4.grid(True)

        # Légendes combinées
        lines = line1 + line2
        labels = [l.get_label() for l in lines]
        ax4.legend(lines, labels, loc='upper right')

        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, 'performance_analysis.png'), dpi=300, bbox_inches='tight')
        plt.show()

        # Figure 2: Analyse spectrale et métriques
        fig2, ((ax5, ax6), (ax7, ax8)) = plt.subplots(2, 2, figsize=fig_size)

        # FFT de l\'erreur de position
        fft_pos = np.fft.fft(self.data['dist_error'])
        freqs = np.fft.fftfreq(len(fft_pos), d=1/self.metrics['sample_rate'])
        ax5.semilogy(freqs[:len(freqs)//2], np.abs(fft_pos[:len(fft_pos)//2]))
        ax5.set_xlabel('Fréquence (Hz)')
        ax5.set_ylabel('Amplitude')
        ax5.set_title('Spectre de l\'Erreur de Position')
        ax5.grid(True)

        # Histogramme des erreurs
        ax6.hist(self.data['dist_error'], bins=30, alpha=0.7, color='blue', edgecolor='black')
        ax6.set_xlabel('Erreur de distance (m)')
        ax6.set_ylabel('Fréquence')
        ax6.set_title('Distribution des Erreurs de Position')
        ax6.grid(True)

        # Évolution des métriques (fenêtre glissante)
        window_size = min(50, len(self.data)//10)
        rolling_rmse = pd.Series(self.data['dist_error']).rolling(window=window_size).apply(lambda x: np.sqrt(np.mean(x**2)))
        ax7.plot(self.data['time'], rolling_rmse, 'purple', linewidth=2)
        ax7.set_xlabel('Temps (s)')
        ax7.set_ylabel('RMSE glissant (m)')
        ax7.set_title(f'RMSE Glissant (fenêtre: {window_size} échantillons)')
        ax7.grid(True)

        # Métriques résumées (texte)
        ax8.axis('off')
        metrics_text = f"""
        MÉTRIQUES DE PERFORMANCE

        Erreur de Position:
        • RMSE: {self.metrics['rmse_position']:.4f} m
        • MAE: {self.metrics['mae_position']:.4f} m
        • Erreur max: {self.metrics['max_error_position']:.4f} m
        • Erreur finale: {self.metrics['final_error_position']:.4f} m

        Erreur d\'Orientation:
        • RMSE: {np.degrees(self.metrics['rmse_orientation']):.2f}°
        • MAE: {np.degrees(self.metrics['mae_orientation']):.2f}°
        • Erreur max: {np.degrees(self.metrics['max_error_orientation']):.2f}°

        Temps de Réponse:
        • Stabilisation position: {self.metrics['settling_time_pos']:.2f} s
        • Stabilisation angle: {self.metrics['settling_time_angle']:.2f} s

        Commandes:
        • Vitesse moyenne: {self.metrics['mean_cmd_vel']:.3f} m/s
        • Vitesse max: {self.metrics['max_cmd_vel']:.3f} m/s
        • ω moyen: {self.metrics['mean_cmd_omega']:.3f} rad/s

        Efficacité:
        • Index énergétique: {self.metrics['energy_index']:.4f}
        • Durée totale: {self.metrics['total_time']:.2f} s
        """
        ax8.text(0.1, 0.9, metrics_text, transform=ax8.transAxes, fontsize=10, 
                verticalalignment='top', fontfamily='monospace',
                bbox=dict(boxstyle='round', facecolor='lightgray', alpha=0.8))

        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, 'detailed_analysis.png'), dpi=300, bbox_inches='tight')
        plt.show()

    def export_metrics(self, output_file: str):
        """Exporte les métriques au format CSV."""
        if not self.metrics:
            return

        metrics_df = pd.DataFrame([self.metrics])
        metrics_df.to_csv(output_file, index=False)
        print(f"Métriques exportées vers: {output_file}")

    def print_summary(self):
        """Affiche un résumé des performances."""
        if not self.metrics:
            return

        print("\n" + "="*50)
        print("RÉSUMÉ DES PERFORMANCES DU CONTRÔLEUR PID")
        print("="*50)

        print(f"\n📊 ERREURS DE POSITION:")
        print(f"   RMSE: {self.metrics['rmse_position']:.4f} m")
        print(f"   Erreur finale: {self.metrics['final_error_position']:.4f} m")

        print(f"\n🎯 ERREURS D\'ORIENTATION:")
        print(f"   RMSE: {np.degrees(self.metrics['rmse_orientation']):.2f}°")
        print(f"   Erreur finale: {np.degrees(self.metrics['final_error_orientation']):.2f}°")

        print(f"\n⏱️ TEMPS DE RÉPONSE:")
        print(f"   Stabilisation position: {self.metrics['settling_time_pos']:.2f} s")
        print(f"   Stabilisation orientation: {self.metrics['settling_time_angle']:.2f} s")

        print(f"\n⚡ EFFICACITÉ:")
        print(f"   Index énergétique: {self.metrics['energy_index']:.4f}")
        print(f"   Vitesse moyenne: {self.metrics['mean_cmd_vel']:.3f} m/s")

        # Évaluation qualitative
        if self.metrics['rmse_position'] < 0.1:
            print(f"\n✅ ÉVALUATION: Excellente précision de position")
        elif self.metrics['rmse_position'] < 0.2:
            print(f"\n✅ ÉVALUATION: Bonne précision de position")
        else:
            print(f"\n⚠️ ÉVALUATION: Précision de position à améliorer")

def main():
    """Fonction principale."""
    parser = argparse.ArgumentParser(description='Analyse des performances du PID turtlesim')
    parser.add_argument('data_file', help='Fichier CSV des données')
    parser.add_argument('--output-dir', default='.', help='Répertoire de sortie')
    parser.add_argument('--export-metrics', help='Fichier d\'export des métriques')

    args = parser.parse_args()

    if not os.path.exists(args.data_file):
        print(f"Erreur: Fichier {args.data_file} introuvable")
        return 1

    analyzer = PIDPerformanceAnalyzer(args.data_file)

    if not analyzer.load_data():
        print("Erreur lors du chargement des données")
        return 1

    analyzer.calculate_metrics()
    analyzer.print_summary()
    analyzer.generate_plots(args.output_dir)

    if args.export_metrics:
        analyzer.export_metrics(args.export_metrics)

    return 0

if __name__ == '__main__':
    sys.exit(main())
'''

# Créer le dossier scripts et écrire le fichier
os.makedirs('~/ros2_ws/src/ros2_pid_turtle/scripts', exist_ok=True)

with open('~/ros2_ws/src/ros2_pid_turtle/scripts/analyze_performance.py', 'w') as f:
    f.write(analysis_script_content)

# Rendre le script exécutable
import stat
st = os.stat('~/ros2_ws/src/ros2_pid_turtle/scripts/analyze_performance.py')
os.chmod('~/ros2_ws/src/ros2_pid_turtle/scripts/analyze_performance.py', st.st_mode | stat.S_IEXEC)

print("Script d'analyse créé avec succès!")



## 8. Checklist Livrables Rapport ROS2

### 8.1 Tableau des Livrables Obligatoires

| Catégorie | Livrable | Format | Status | Description |
|:----------|:---------|:-------|:-------|:------------|
| **📁 Code Source** | ||||
|| `package.xml` | XML | ✅ | Métadonnées et dépendances |
|| `setup.py` | Python | ✅ | Configuration installation |
|| `control_node.py` | Python | ✅ | Node principal PID |
|| `pid_controller.py` | Python | ✅ | Classe contrôleur PID |
|| `bicycle_model.py` | Python | ✅ | Modèle cinématique |
|| `trajectory_generator.py` | Python | ✅ | Générateur trajectoires |
| **⚙️ Configuration** | ||||
|| `pid_params.yaml` | YAML | ✅ | Paramètres PID |
|| `trajectory_params.yaml` | YAML | ✅ | Paramètres trajectoires |
|| `pid_control.launch.py` | Python | ✅ | Launch principal |
|| `analysis.launch.py` | Python | ✅ | Launch analyse |
| **📊 Analyse** | ||||
|| `analyze_performance.py` | Python | ✅ | Script analyse performance |
|| Données expérimentales | CSV/Rosbag | ⏳ | À générer |
|| Graphiques de performance | PNG | ⏳ | À générer |
|| Métriques de performance | CSV | ⏳ | À générer |
| **📝 Documentation** | ||||
|| Notebook Jupyter | IPYNB | ✅ | Ce document |
|| README.md | Markdown | ⏳ | Instructions utilisation |
|| Rapport technique | PDF | ⏳ | Rapport final |

### 8.2 Instructions pour Générer les Livrables

#### Phase 1: Compilation et Test
```bash
# 1. Compilation du projet
cd ~/ros2_ws
colcon build --packages-select ros2_pid_turtle
source install/setup.bash

# 2. Test de fonctionnement
ros2 launch ros2_pid_turtle pid_control.launch.py target_x:=8.0 target_y:=8.0
```

#### Phase 2: Collecte de Données
```bash
# 3. Expérience contrôlée (60 secondes)
ros2 launch ros2_pid_turtle analysis.launch.py experiment_duration:=60 bag_file:=/tmp/exp_circle

# 4. Différents types de trajectoires
ros2 launch ros2_pid_turtle analysis.launch.py trajectory_type:=square experiment_duration:=45
ros2 launch ros2_pid_turtle analysis.launch.py trajectory_type:=figure8 experiment_duration:=90
```

#### Phase 3: Analyse et Graphiques
```bash
# 5. Conversion rosbag vers CSV
ros2 bag play /tmp/exp_circle &
ros2 topic echo /control_data --csv > experiment_data.csv

# 6. Génération des graphiques et métriques
python3 ~/ros2_ws/src/ros2_pid_turtle/scripts/analyze_performance.py experiment_data.csv --output-dir ./results --export-metrics metrics.csv
```

#### Phase 4: Captures d'Écran et Monitoring
```bash
# 7. Captures rqt_graph
rqt_graph  # Capture manuelle: graph_system.png

# 8. Captures monitoring
rqt_plot /turtle1/pose/x,/turtle1/pose/y  # Capture: trajectory_plot.png
rqt_plot /control_data  # Capture: control_signals.png

# 9. Logs et debugging
ros2 run ros2_pid_turtle control_node --ros-args --log-level DEBUG > debug_logs.txt 2>&1
```

### 8.3 Structure des Fichiers de Rendu

```
📁 RENDU_NOM_PRENOM/
├── 📁 src/
│   └── 📁 ros2_pid_turtle/          # Package complet
├── 📁 data/
│   ├── experiment_circle.bag        # Données rosbag
│   ├── experiment_square.bag
│   ├── experiment_data.csv          # Données CSV
│   └── metrics.csv                  # Métriques performance
├── 📁 results/
│   ├── performance_analysis.png     # Graphiques analyse
│   ├── detailed_analysis.png
│   ├── graph_system.png            # Captures rqt_graph
│   └── trajectory_plot.png         # Captures monitoring
├── 📁 docs/
│   ├── ros2_pid_notebook.ipynb     # Ce notebook
│   ├── README.md                   # Instructions
│   └── rapport_technique.pdf       # Rapport final
└── 📁 logs/
    ├── debug_logs.txt              # Logs debug
    ├── build_output.txt            # Logs compilation
    └── test_results.txt            # Résultats tests
```

### 8.4 Critères d'Évaluation

| Critère | Points | Évaluation |
|:--------|:-------|:-----------|
| **Fonctionnalité** (40%) |||
| Compilation sans erreur | 10 | Package se compile proprement |
| Exécution du contrôleur | 15 | Node fonctionne et contrôle la tortue |
| Atteinte des objectifs | 15 | Tortue atteint la cible avec précision |
| **Qualité du Code** (30%) |||
| Structure et organisation | 10 | Code bien organisé et commenté |
| Respect des standards ROS2 | 10 | Bonnes pratiques ROS2 |
| Gestion des erreurs | 10 | Robustesse et gestion exceptions |
| **Analyse et Documentation** (30%) |||
| Analyse des performances | 15 | Métriques et graphiques pertinents |
| Documentation complète | 10 | README, commentaires, rapport |
| Reproductibilité | 5 | Instructions claires et complètes |

### 8.5 Check-list Finale

- [ ] ✅ Compilation réussie du package
- [ ] ✅ Exécution du contrôleur PID  
- [ ] ✅ Génération des trajectoires
- [ ] ✅ Enregistrement rosbag des expériences
- [ ] ✅ Analyse des données et métriques
- [ ] ✅ Génération des graphiques
- [ ] ✅ Captures d'écran du système
- [ ] ✅ Documentation complète
- [ ] ✅ Tests de reproductibilité
- [ ] ✅ Archivage des livrables

**🎯 Objectif:** Démontrer la maîtrise des concepts d'automatique (PID, modèle bicyclette) et de ROS2 (nodes, topics, launch, paramètres) à travers un projet industriel reproductible.
